In [60]:
%load_ext autoreload

%autoreload 2

import configparser
import os
import time
import random
import pandas as pd
import requests
import openpyxl
import sqlite3
from sklearn.model_selection import train_test_split

from datetime import datetime
from io import StringIO
from pathlib import Path
from bs4 import BeautifulSoup
from alive_progress import alive_bar

# some stuff I set up in a config file so I don't have to keep updating certain
# variables in every script
config = configparser.ConfigParser()
config.read('../src/config.ini')
output = Path(config['paths']['output'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
data = pd.read_csv(config['data']['model_data'])
upcoming_games = pd.read_csv(r'')
upcoming_game.columns = data.columns

In [62]:
upcoming_game

,index,game_date,day,round,home_team,away_team,result,overall_result,home_score,home_penalties,...,away_prior_SCA,away_prior_GCA,away_prior_Cmp,away_prior_Att,away_prior_Cmp%,away_prior_PrgP,away_prior_Carries,away_prior_PrgC,away_prior_Att_TakeOn,away_prior_Succ
0,0,2024-11-24,Sat,Conference Semifinals,Orlando City,Atlanta Utd,NaN,NaN,0,0,...,2.171717,0.217172,34.919192,42.282828,81.000505,3.515152,28.777778,1.606061,1.272727,0.590909


In [63]:
for col in data.columns:
    print(col)

index
game_date
day
round
home_team
away_team
result
overall_result
home_score
home_penalties
away_score
away_penalties
game_key
index.1
home_prior_Min
home_prior_Gls
home_prior_Ast
home_prior_PK
home_prior_PKatt
home_prior_Sh
home_prior_SoT
home_prior_CrdY
home_prior_CrdR
home_prior_Touches
home_prior_Tkl
home_prior_Int
home_prior_Blocks
home_prior_xG
home_prior_npxG
home_prior_xAG
home_prior_SCA
home_prior_GCA
home_prior_Cmp
home_prior_Att
home_prior_Cmp%
home_prior_PrgP
home_prior_Carries
home_prior_PrgC
home_prior_Att_TakeOn
home_prior_Succ
index.2
away_prior_Min
away_prior_Gls
away_prior_Ast
away_prior_PK
away_prior_PKatt
away_prior_Sh
away_prior_SoT
away_prior_CrdY
away_prior_CrdR
away_prior_Touches
away_prior_Tkl
away_prior_Int
away_prior_Blocks
away_prior_xG
away_prior_npxG
away_prior_xAG
away_prior_SCA
away_prior_GCA
away_prior_Cmp
away_prior_Att
away_prior_Cmp%
away_prior_PrgP
away_prior_Carries
away_prior_PrgC
away_prior_Att_TakeOn
away_prior_Succ


In [64]:
data.drop(columns=['index','game_date', 'result', 'game_key', 'index.1', 'index.2'], inplace=True)

In [65]:
upcoming_game.drop(columns=['index','game_date', 'result', 'game_key', 'index.1', 'index.2'], inplace=True)

In [66]:
X = data.drop(columns=['overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties'])
upcoming_game = upcoming_game.drop(columns=['overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties'])
y = data[['overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties']]

In [67]:
def result_to_num(result):
    if result == "D":
        return 0
    elif result =="L":
        return -1
    else:
        return 1

In [68]:
y['overall_result'] = y['overall_result'].apply(result_to_num)

C:\Users\Cristian\AppData\Local\Temp\ipykernel_25512\1505406061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['overall_result'] = y['overall_result'].apply(result_to_num)


In [69]:
y

,overall_result,home_score,home_penalties,away_score,away_penalties
0,-1,0,0,1,0
1,-1,1,0,2,0
2,0,1,0,1,0
3,1,1,0,0,0
4,-1,0,0,3,0
...,...,...,...,...,...
1482,-1,0,0,1,0
1483,-1,0,0,1,0
1484,-1,1,0,2,0
1485,0,0,0,0,0


In [70]:
X

,day,round,home_team,away_team,home_prior_Min,home_prior_Gls,home_prior_Ast,home_prior_PK,home_prior_PKatt,home_prior_Sh,...,away_prior_SCA,away_prior_GCA,away_prior_Cmp,away_prior_Att,away_prior_Cmp%,away_prior_PrgP,away_prior_Carries,away_prior_PrgC,away_prior_Att_TakeOn,away_prior_Succ
0,Sun,Regular Season,NE Revolution,Toronto FC,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Sun,Regular Season,NE Revolution,FC Cincinnati,82.636364,0.000000,0.000000,0.000000,0.000000,1.818182,...,2.227273,0.000000,33.227273,41.590909,77.390909,2.500000,26.181818,1.136364,2.272727,1.045455
2,Sat,Regular Season,NE Revolution,Chicago Fire,83.818182,0.045455,0.000000,0.000000,0.000000,1.636364,...,2.045455,0.318182,38.545455,47.227273,81.131818,3.681818,31.272727,1.454545,1.727273,0.590909
3,Sat,Regular Season,NE Revolution,Charlotte,84.969697,0.060606,0.000000,0.000000,0.000000,1.484848,...,1.878788,0.212121,26.636364,34.818182,75.272727,2.757576,21.848485,1.363636,0.787879,0.333333
4,Sat,Regular Season,NE Revolution,Philadelphia Union,84.236364,0.072727,0.018182,0.000000,0.000000,1.200000,...,2.787879,0.181818,30.893939,40.106061,75.646875,3.939394,27.106061,1.530303,0.893939,0.393939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,Sat,Regular Season,Sporting KC,Vancouver W'caps,83.166667,0.106061,0.030303,0.015152,0.015152,0.924242,...,1.651515,0.166667,27.363636,36.015152,75.268182,3.348485,23.484848,1.166667,0.893939,0.500000
1483,Sun,Regular Season,Sporting KC,NY Red Bulls,81.420455,0.090909,0.022727,0.011364,0.011364,0.943182,...,1.959596,0.191919,25.606061,36.373737,69.514141,3.272727,22.616162,1.333333,1.373737,0.707071
1484,Sat,Regular Season,Sporting KC,Nashville SC,86.590909,0.090909,0.000000,0.000000,0.000000,1.181818,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1485,Sat,Regular Season,Sporting KC,Columbus Crew,85.242424,0.090909,0.030303,0.000000,0.000000,1.181818,...,2.416667,0.187500,39.562500,48.000000,80.537500,4.354167,37.958333,2.083333,1.750000,0.854167


In [71]:
from sklearn.tree import DecisionTreeRegressor

In [72]:
from sklearn.preprocessing import OneHotEncoder

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=42)

In [74]:
X_train

,day,round,home_team,away_team,home_prior_Min,home_prior_Gls,home_prior_Ast,home_prior_PK,home_prior_PKatt,home_prior_Sh,...,away_prior_SCA,away_prior_GCA,away_prior_Cmp,away_prior_Att,away_prior_Cmp%,away_prior_PrgP,away_prior_Carries,away_prior_PrgC,away_prior_Att_TakeOn,away_prior_Succ
363,Sat,Regular Season,Nashville SC,CF Montréal,83.469697,0.121212,0.090909,0.015152,0.015152,0.848485,...,1.636364,0.227273,40.772727,49.500000,81.500000,3.181818,29.272727,1.500000,1.590909,0.954545
1167,Sun,Regular Season,Nashville SC,LAFC,84.022727,0.113636,0.079545,0.011364,0.011364,1.102273,...,2.306452,0.314516,33.266129,42.483871,76.143548,4.370968,30.177419,1.580645,1.903226,0.967742
1187,Sat,Regular Season,Toronto FC,Seattle Sounders FC,80.383929,0.142857,0.098214,0.017857,0.017857,0.758929,...,1.898990,0.242424,36.787879,45.414141,79.723232,3.353535,30.666667,1.060606,1.040404,0.636364
346,Sat,Regular Season,FC Dallas,LA Galaxy,82.504132,0.157025,0.090909,0.008264,0.008264,1.173554,...,2.305785,0.330579,44.471074,51.528926,84.357025,3.975207,33.966942,1.685950,1.661157,0.785124
1346,Wed,Regular Season,Houston Dynamo,Seattle Sounders FC,80.779221,0.116883,0.064935,0.012987,0.012987,0.870130,...,1.681818,0.227273,34.318182,43.068182,77.872727,3.250000,30.522727,1.090909,1.000000,0.568182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,Tue,Regular Season,Sporting KC,D.C. United,81.629870,0.110390,0.045455,0.006494,0.012987,1.045455,...,1.634831,0.168539,26.528090,35.741573,72.748315,2.882022,21.516854,1.134831,1.174157,0.511236
1294,Sat,Regular Season,Orlando City,Charlotte,81.428571,0.107143,0.089286,0.000000,0.000000,1.017857,...,1.543478,0.260870,35.282609,43.673913,79.565217,2.804348,31.217391,0.760870,1.108696,0.608696
860,Sat,Regular Season,FC Dallas,Columbus Crew,82.720779,0.116883,0.103896,0.000000,0.000000,0.896104,...,2.460606,0.375758,41.121212,48.212121,83.607879,4.345455,36.339394,2.030303,2.103030,1.060606
1459,Sat,Regular Season,Austin,FC Cincinnati,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [75]:
y_train

,overall_result,home_score,home_penalties,away_score,away_penalties
363,1,4,0,1,0
1167,-1,1,0,2,0
1187,-1,0,0,2,0
346,1,2,0,0,0
1346,-1,0,0,1,0
...,...,...,...,...,...
1130,1,3,0,0,0
1294,1,2,0,1,0
860,0,1,0,1,0
1459,1,5,0,0,0


In [76]:
X_train_cats = X_train.select_dtypes(include='O')
X_test_cats = X_test.select_dtypes(include='O')
upcoming_cats = upcoming_game.select_dtypes(include='O')

In [77]:
X_train_cats

,day,round,home_team,away_team
363,Sat,Regular Season,Nashville SC,CF Montréal
1167,Sun,Regular Season,Nashville SC,LAFC
1187,Sat,Regular Season,Toronto FC,Seattle Sounders FC
346,Sat,Regular Season,FC Dallas,LA Galaxy
1346,Wed,Regular Season,Houston Dynamo,Seattle Sounders FC
...,...,...,...,...
1130,Tue,Regular Season,Sporting KC,D.C. United
1294,Sat,Regular Season,Orlando City,Charlotte
860,Sat,Regular Season,FC Dallas,Columbus Crew
1459,Sat,Regular Season,Austin,FC Cincinnati


In [78]:
encoder = OneHotEncoder()

In [79]:
encoder.fit(X_train_cats)
X_train_cats_enc = encoder.transform(X_train_cats).toarray()
X_test_cats_enc = encoder.transform(X_test_cats).toarray()
upcoming_cats_enc = encoder.transform(upcoming_cats).toarray()

In [80]:
X_train_cats_enc = pd.DataFrame(X_train_cats_enc)
X_test_cats_enc = pd.DataFrame(X_test_cats_enc)
upcoming_cats_enc = pd.DataFrame(upcoming_cats_enc)

In [81]:
X_train_cats_enc

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1111,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1112,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1113,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
X_train_cats_enc.columns = encoder.get_feature_names_out()
X_train_cats_enc.index = X_train_cats.index
X_test_cats_enc.columns = encoder.get_feature_names_out()
X_test_cats_enc.index = X_test_cats.index
upcoming_cats_enc.columns = encoder.get_feature_names_out()
upcoming_cats_enc.index = upcoming_cats.index

In [83]:
X_train_nums = X_train.select_dtypes(exclude='O')
X_test_nums = X_test.select_dtypes(exclude='O')
upcoming_nums = upcoming_game.select_dtypes(exclude='O')

In [84]:
X_train_final = pd.concat([X_train_nums, X_train_cats_enc],axis=1)
X_test_final = pd.concat([X_test_nums, X_test_cats_enc],axis=1)
upcoming_final = pd.concat([upcoming_nums, upcoming_cats_enc], axis=1)

In [85]:
X_train_final

,home_prior_Min,home_prior_Gls,home_prior_Ast,home_prior_PK,home_prior_PKatt,home_prior_Sh,home_prior_SoT,home_prior_CrdY,home_prior_CrdR,home_prior_Touches,...,away_team_Orlando City,away_team_Philadelphia Union,away_team_Portland Timbers,away_team_Real Salt Lake,away_team_SJ Earthquakes,away_team_Seattle Sounders FC,away_team_Sporting KC,away_team_St. Louis,away_team_Toronto FC,away_team_Vancouver W'caps
363,83.469697,0.121212,0.090909,0.015152,0.015152,0.848485,0.333333,0.121212,0.000000,46.424242,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1167,84.022727,0.113636,0.079545,0.011364,0.011364,1.102273,0.477273,0.147727,0.000000,48.147727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1187,80.383929,0.142857,0.098214,0.017857,0.017857,0.758929,0.357143,0.160714,0.000000,55.526786,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
346,82.504132,0.157025,0.090909,0.008264,0.008264,1.173554,0.413223,0.140496,0.000000,50.082645,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1346,80.779221,0.116883,0.064935,0.012987,0.012987,0.870130,0.311688,0.220779,0.025974,46.857143,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,81.629870,0.110390,0.045455,0.006494,0.012987,1.045455,0.344156,0.175325,0.025974,48.279221,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1294,81.428571,0.107143,0.089286,0.000000,0.000000,1.017857,0.357143,0.214286,0.017857,54.642857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,82.720779,0.116883,0.103896,0.000000,0.000000,0.896104,0.233766,0.136364,0.000000,48.279221,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
y_train

,overall_result,home_score,home_penalties,away_score,away_penalties
363,1,4,0,1,0
1167,-1,1,0,2,0
1187,-1,0,0,2,0
346,1,2,0,0,0
1346,-1,0,0,1,0
...,...,...,...,...,...
1130,1,3,0,0,0
1294,1,2,0,1,0
860,0,1,0,1,0
1459,1,5,0,0,0


In [87]:
dtr = DecisionTreeRegressor(random_state=42)
dtr.fit(X_train_final,y_train)

DecisionTreeRegressor(random_state=42)

In [88]:
preds = pd.DataFrame(dtr.predict(X_test_final))

In [89]:
preds.columns = y_test.columns
preds

,overall_result,home_score,home_penalties,away_score,away_penalties
0,1.0,4.0,0.0,2.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,-1.0,2.0,0.0,3.0,0.0
3,1.0,2.0,0.0,1.0,0.0
4,1.0,3.0,0.0,2.0,0.0
...,...,...,...,...,...
367,0.0,0.0,0.0,0.0,0.0
368,1.0,3.0,0.0,2.0,0.0
369,1.0,2.0,0.0,0.0,0.0
370,0.0,1.0,0.0,1.0,0.0


In [90]:
X_test_final[['overall_result_pred', 'home_score_pred', 'home_penalties_pred', 'away_score_pred', 'away_penalties_pred']] = preds
X_test_final[['overall_result', 'home_score', 'home_penalties', 'away_score', 'away_penalties']] = y_test

In [91]:
X_test_final

,home_prior_Min,home_prior_Gls,home_prior_Ast,home_prior_PK,home_prior_PKatt,home_prior_Sh,home_prior_SoT,home_prior_CrdY,home_prior_CrdR,home_prior_Touches,...,overall_result_pred,home_score_pred,home_penalties_pred,away_score_pred,away_penalties_pred,overall_result,home_score,home_penalties,away_score,away_penalties
1317,82.991736,0.107438,0.099174,0.000000,0.000000,1.115702,0.322314,0.157025,0.000000,50.991736,...,NaN,NaN,NaN,NaN,NaN,1,1,0,0,0
1455,81.828358,0.104478,0.104478,0.014925,0.014925,0.970149,0.305970,0.186567,0.007463,47.373134,...,NaN,NaN,NaN,NaN,NaN,1,3,0,2,0
439,81.040816,0.129252,0.136054,0.006803,0.006803,1.088435,0.346939,0.170068,0.013605,56.081633,...,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0
560,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,1,2,0,1,0
433,83.913580,0.098765,0.111111,0.012346,0.012346,1.135802,0.358025,0.135802,0.012346,57.604938,...,NaN,NaN,NaN,NaN,NaN,1,5,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,80.888889,0.080808,0.030303,0.030303,0.030303,0.929293,0.282828,0.181818,0.000000,47.979798,...,NaN,NaN,NaN,NaN,NaN,1,2,0,1,0
1234,79.615385,0.118881,0.076923,0.020979,0.027972,1.097902,0.377622,0.160839,0.006993,51.615385,...,NaN,NaN,NaN,NaN,NaN,0,3,0,3,0
1157,79.778409,0.142045,0.102273,0.022727,0.034091,1.051136,0.392045,0.147727,0.011364,51.681818,...,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0
916,83.959596,0.146465,0.111111,0.025253,0.025253,1.111111,0.343434,0.191919,0.015152,54.429293,...,NaN,NaN,NaN,NaN,NaN,0,1,4,1,3


In [92]:
upcoming_pred = pd.DataFrame(dtr.predict(upcoming_final))

In [93]:
upcoming_pred.columns = y_test.columns

In [94]:
upcoming_pred

,overall_result,home_score,home_penalties,away_score,away_penalties
0,-1.0,3.0,0.0,4.0,0.0


In [95]:
upcoming_game[['overall_result_pred', 'home_score_pred', 'home_penalties_pred', 'away_score_pred', 'away_penalties_pred']] = upcoming_pred

In [96]:
upcoming_game

,day,round,home_team,away_team,home_prior_Min,home_prior_Gls,home_prior_Ast,home_prior_PK,home_prior_PKatt,home_prior_Sh,...,away_prior_PrgP,away_prior_Carries,away_prior_PrgC,away_prior_Att_TakeOn,away_prior_Succ,overall_result_pred,home_score_pred,home_penalties_pred,away_score_pred,away_penalties_pred
0,Sat,Conference Semifinals,Orlando City,Atlanta Utd,82.244019,0.119617,0.08134,0.009569,0.019139,1.177033,...,3.515152,28.777778,1.606061,1.272727,0.590909,-1.0,3.0,0.0,4.0,0.0


In [97]:
upcoming_pred

,overall_result,home_score,home_penalties,away_score,away_penalties
0,-1.0,3.0,0.0,4.0,0.0


In [98]:
from sklearn.metrics import mean_absolute_error

In [99]:
mean_absolute_error(y_test, preds)

np.float64(0.6580645161290322)

In [100]:
from joblib import dump, load

In [101]:
# dump(dtr, output/'dtr_mvp.joblib')
# dtr_test = load(output / 'dtr_mvp.joblib')

In [102]:
dtr_test.predict(upcoming_final)

NameError: name 'dtr_test' is not defined

In [ ]:
# dump(encoder, output/'encoder_mvp.joblib')